In [63]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression

In [38]:
df= pd.read_csv("iphone6-negative.csv",encoding="latin-1")
df.head()

,TweetID,User_ID,Text,Sentiment
0,6.473990e+17,Rionagh,I've had an IPhone for like 3 years and I've n...,negative
1,6.473990e+17,your highness,I dont need the new iphone but I want it :(,negative
2,6.473990e+17,Little Liggins,fuck the iPhone 6s cus I'm not getting one :(,negative
3,6.474000e+17,NaN,Hopefully will be able to get my iPhone 6s tod...,negative
4,6.474000e+17,Kenny Tosh,@clydesdalebank I've just bought a new iPhone ...,negative


In [39]:
df = df[['Text','Sentiment']]
df.head()

,Text,Sentiment
0,I've had an IPhone for like 3 years and I've n...,negative
1,I dont need the new iphone but I want it :(,negative
2,fuck the iPhone 6s cus I'm not getting one :(,negative
3,Hopefully will be able to get my iPhone 6s tod...,negative
4,@clydesdalebank I've just bought a new iPhone ...,negative


In [41]:
def cleaner(text):
    text = text.lower()
    text = re.sub("@[^\s]+","",text)
    text = text.replace(":)","")  
    text = text.replace("@","")  
    text = text.replace(":(","")
    return text

In [42]:
def remove_stop_words(text):
    sw = stopwords.words("english")
    clean_words = []
    text = text.split()
    for word in text:
        if word not in sw:
            clean_words.append(word)
    return " ".join(clean_words)

In [43]:
def stemming(text):
    ps = PorterStemmer()
    text = text.split()
    stemmed_words = []
    for word in text :
        stemmed_words.append(ps.stem(word))
    return " ".join(stemmed_words)

In [44]:
def run(text):
    text = cleaner(text)
    text = remove_stop_words(text)
    text = stemming(text)
    return text

In [45]:
df['Text'] = df['Text'].apply(run)

In [46]:
df.head()

,Text,Sentiment
0,i'v iphon like 3 year i'v never done hate life,negative
1,dont need new iphon want,negative
2,fuck iphon 6s cu i'm get one,negative
3,hope abl get iphon 6s today plane get like 11:...,negative
4,i'v bought new iphon can't use appl pay suppor...,negative


In [52]:
tfidf = TfidfVectorizer()
x = tfidf.fit_transform(df["Text"]).toarray()


In [61]:
df['Sentiment'] = df['Sentiment'].replace("negative",0)
df['Sentiment'] = df['Sentiment'].replace("positive",1)


In [62]:
y = df['Sentiment'].values
y

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [64]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25)

In [65]:
lr = LogisticRegression()
lr.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [66]:
y_pred = lr.predict(x_test)

In [67]:
lr.score(x_test,y_test)

0.8844621513944223

In [106]:
test = "Hi I'm Here to Slay!  : SayDrizzy Wanna get FREE iPhone 6? Kindly check my bi0 :) Thx https://t.co/jB7x2VDqdj"
test = run(test)
test = tfidf.transform([test]).toarray()

In [107]:
lr.predict(test)

array([1], dtype=int64)

In [104]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

             precision    recall  f1-score   support

          0       1.00      0.82      0.90       157
          1       0.76      1.00      0.87        94

avg / total       0.91      0.88      0.89       251



In [105]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred,y_test)


array([[128,  29],
       [  0,  94]], dtype=int64)